## import

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
#LWM을 하기위한 라이브러리 가져오기
import DeepMIMOv3
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
import time


plt . rcParams [ 'figure.figsize' ]  =  [ 12 ,  8 ]  # 기본 플롯 크기 설정

## GPU설정

In [2]:
# GPU 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
import torch
print(torch.version.cuda)                   # 설치된 CUDA 버전 (예: '11.7')
print(torch.backends.cudnn.version())       # cuDNN 버전 (예: 8200)
print("CUDA available:", torch.cuda.is_available())  # True


11.8
90100
CUDA available: True


# DeepMIMOv3 다운

In [4]:
# pip install DeepMIMOv3 umap-learn

## 파라미터 수정

In [5]:
## Load and print the default parameters
# bandwith: 0.05GHz(50MHz 대역폭 사용)
parameters = DeepMIMOv3.default_params()
pprint(parameters)

{'OFDM': {'RX_filter': 0,
          'bandwidth': 0.05,
          'selected_subcarriers': array([0]),
          'subcarriers': 512},
 'OFDM_channels': 1,
 'active_BS': array([1]),
 'bs_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([8, 4]),
                'spacing': 0.5},
 'dataset_folder': './Raytracing_scenarios',
 'dynamic_scenario_scenes': array([1]),
 'enable_BS2BS': 1,
 'enable_doppler': 0,
 'enable_dual_polar': 0,
 'num_paths': 5,
 'scenario': 'O1_60',
 'ue_antenna': {'FoV': array([360, 180]),
                'radiation_pattern': 'isotropic',
                'rotation': array([0, 0, 0]),
                'shape': array([4, 2]),
                'spacing': 0.5},
 'user_rows': array([1]),
 'user_subsampling': 1}


In [6]:
## Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
#LWM 동적 시나리오 불러오기
#자신의 LWM 파일 위치 경로 작성
# parameters['dataset_folder'] = r'/content/drive/MyDrive/Colab Notebooks/LWM'
scene = 15 # 장면 수
parameters['dataset_folder'] = r'C:\Users\dlghd\졸업프로젝트\LWM'

# scnario = 02_dyn_3p5 <- 다운받은 파일(동적시나리오)
parameters['scenario'] = 'O2_dyn_3p5'
parameters['dynamic_scenario_scenes'] = np.arange(scene) #scene 0~9

# 각 사용자-기지국 채널에 대해 최대 10개 멀티패스 경로 사용
parameters['num_paths'] = 10

# User rows 1-100
parameters['user_rows'] = np.arange(100)
# User 축소하기
parameters['user_subsampling'] = 0.01

# Activate only the first basestation
parameters['active_BS'] = np.array([1])

parameters['activate_OFDM'] = 1

parameters['OFDM']['bandwidth'] = 0.05 # 50 MHz
parameters['OFDM']['subcarriers'] = 512 # OFDM with 512 subcarriers
parameters['OFDM']['selected_subcarriers'] = np.arange(0, 64, 1)
#parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['ue_antenna']['shape'] = np.array([1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32]) # ULA of 32 elements
#parameters['bs_antenna']['rotation'] = np.array([0, 30, 90]) # ULA of 32 elements
#parameters['ue_antenna']['rotation'] = np.array([[0, 30], [30, 60], [60, 90]]) # ULA of 32 elements
#parameters['ue_antenna']['radiation_pattern'] = 'isotropic'
#parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'

In [7]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]), 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 0.01, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable

## dataset 구축

In [8]:
## dataset 구축 (chunked on‑the‑fly generation)
import time, gc
from tqdm import tqdm

# 0~999 씬 인덱스, 한 번에 50개씩 처리
scene_indices = np.arange(scene)
chunk_size   = 5
all_data     = []

# 씬 묶음(chunk)마다 generate_data 호출
for i in tqdm(range(0, len(scene_indices), chunk_size)):
    chunk = scene_indices[i : i+chunk_size].tolist()
    parameters['dynamic_scenario_scenes'] = chunk

    start = time.time()
    data_chunk = DeepMIMOv3.generate_data(parameters)
    print(f"Scenes {chunk[0]}–{chunk[-1]} generation time: {time.time() - start:.2f}s")

    # 바로 all_data에 합치거나, 디스크에 저장해도 OK
    all_data.extend(data_chunk)

    # 메모리 해제
    del data_chunk
    gc.collect()

# 마지막에 하나의 리스트로 합친 데이터셋
dataset = all_data


  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

The following parameters seem unnecessary:
{'activate_OFDM'}

Scene 1/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 202024.59it/s]

Generating channels: 100%|█████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 4967.50it/s]



BS-BS Channels



Reading ray-tracing: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1000.55it/s]

Generating channels: 100%|██████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 368.83it/s]


...


Scene 5/5

Basestation 1

UE-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████| 69006/69006 [00:00<00:00, 178003.31it/s]

Generating channels: 100%|█████████████████████████████████████████████████████████| 727/727 [00:00<00:00, 4255.95it/s]



BS-BS Channels



Reading ray-tracing: 100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:24<00:00,  8.12s/it]

Scenes 10–14 generation time: 7.95s


In [9]:
print(parameters['user_rows'])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [10]:
print(parameters)

{'dataset_folder': 'C:\\Users\\dlghd\\졸업프로젝트\\LWM', 'scenario': 'O2_dyn_3p5', 'dynamic_scenario_scenes': [10, 11, 12, 13, 14], 'num_paths': 10, 'active_BS': array([1]), 'user_rows': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]), 'user_subsampling': 0.01, 'bs_antenna': {'shape': array([ 1, 32]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'ue_antenna': {'shape': array([1, 1]), 'spacing': 0.5, 'rotation': array([0, 0, 0]), 'FoV': array([360, 180]), 'radiation_pattern': 'isotropic'}, 'enable_doppler': 0, 'enable_dual_polar': 0, 'enable_B

# 사용자 접근 데이터

In [11]:
user_data = dataset[0][0]['user']
print(user_data.keys())

dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# 사용자 채널 정보 확인

In [12]:
# subcarries = 나눈 각각의 주파수 채널
# Channel = H <- 채널 벡터
# 채널 형태
# (user, UE antenna, Bs antenna, subcarrier)
channel = dataset[0][0]['user']['channel']
print(channel.shape)  

(727, 1, 32, 64)


In [13]:
print(dataset[0][0]['user']['channel'][100])

[[[ 8.57045598e-06+5.5781261e-06j  8.89099283e-06+5.0515800e-06j
    9.17921989e-06+4.5066768e-06j ... -1.02173499e-05-4.1711201e-07j
   -1.02239183e-05+1.9928974e-07j -1.01933329e-05+8.1496728e-07j]
  [ 1.02161603e-05+4.4529790e-07j  1.02244285e-05-1.7108337e-07j
    1.01955429e-05-7.8684292e-07j ... -9.00999748e-06+4.8361312e-06j
   -8.70222630e-06+5.3702393e-06j -8.36283198e-06+5.8848323e-06j]
  [ 9.02330430e-06-4.8112561e-06j  8.71700831e-06-5.3462113e-06j
    8.37903553e-06-5.8617388e-06j ... -5.29921817e-06+8.7456565e-06j
   -4.76262221e-06+9.0490685e-06j -4.20871947e-06+9.3195977e-06j]
  ...
  [-7.00710962e-06-7.4477266e-06j -7.44313866e-06-7.0119827e-06j
   -7.85211978e-06-6.5507575e-06j ...  9.82847632e-06+2.8229874e-06j
    9.98071755e-06+2.2256456e-06j  1.00966881e-05+1.6202162e-06j]
  [-9.82065103e-06-2.8500913e-06j -9.97453935e-06-2.2531719e-06j
   -1.00921798e-05-1.6480645e-06j ...  9.89848286e-06-2.5667589e-06j
    9.72583803e-06-3.1585257e-06j  9.51785023e-06-3.7388147e

# 사용자 위치 정보

In [14]:
location = dataset[0][0]['user']['location']
print(location.shape)      # (사용자 수, 3)
print(location[0:4])         # 첫 번째 사용자의 (x, y, z)

(727, 3)
[[-71.03330231 -15.57629967   1.        ]
 [-68.63330078 -15.57629967   1.        ]
 [-52.83330154 -15.57629967   1.        ]
 [-31.23329926 -15.57629967   1.        ]]


# 경로정보

In [15]:
paths = dataset[0][0]['user']['paths']
#사용자 수
print(len(paths))
# 첫 번째 사용자 경로 정보
print(paths[0])

727
{'num_paths': 2, 'DoD_phi': array([-160.941, -160.941], dtype=float32), 'DoD_theta': array([93.6525, 94.7439], dtype=float32), 'DoA_phi': array([19.0585, 19.0585], dtype=float32), 'DoA_theta': array([86.3475, 94.7439], dtype=float32), 'phase': array([ 143.357, -137.611], dtype=float32), 'ToA': array([2.61886e-07, 2.62253e-07], dtype=float32), 'LoS': array([1., 0.], dtype=float32), 'power': array([7.5363324e-09, 3.2098095e-09], dtype=float32)}


# 기지국 정보

In [16]:
bs_data = dataset[0][0]['basestation']
print(bs_data.keys())


dict_keys(['paths', 'LoS', 'location', 'distance', 'pathloss', 'channel'])


# Scene 및 사용자 수

In [17]:
for i, scene in enumerate(dataset[0]):
    user_locs = scene['user']['location']
    print(f"Scene {i}: {len(user_locs)} users")

Scene 0: 727 users


# 채널 수

In [18]:
len(dataset[0][0]['user']['channel'])

727

In [19]:
print(dataset[0][0]['user']['paths'][0])

{'num_paths': 2, 'DoD_phi': array([-160.941, -160.941], dtype=float32), 'DoD_theta': array([93.6525, 94.7439], dtype=float32), 'DoA_phi': array([19.0585, 19.0585], dtype=float32), 'DoA_theta': array([86.3475, 94.7439], dtype=float32), 'phase': array([ 143.357, -137.611], dtype=float32), 'ToA': array([2.61886e-07, 2.62253e-07], dtype=float32), 'LoS': array([1., 0.], dtype=float32), 'power': array([7.5363324e-09, 3.2098095e-09], dtype=float32)}


In [20]:
scene = dataset[0][0] # scene 0
ue_idx = 0 # 첫 번째 사용자
channel = scene['user']['channel'][ue_idx]
print(channel.shape)

(1, 32, 64)


# channel CIR mat 정보 가져오기

In [21]:
import scipy.io as sio

file_path = r'C:\Users\dlghd\졸업프로젝트\LWM\O2_dyn_3p5\scene_0\O2_dyn_3p5.1.CIR.mat'
mat_data = sio.loadmat(file_path)

# 파일 안의 key 확인
print(mat_data.keys())




dict_keys(['__header__', '__version__', '__globals__', 'CIR_array_full'])


In [22]:
# 일반적으로 CIR key는 'CIR' 또는 'cir' 같은 이름일 가능성 높음
H_cir = mat_data['__header__']  
print(H_cir)

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Jun 30 11:33:01 2021'


# Time-Prediction 시작
## Time Series 형태로 변환
### 단일사용자 채널 예측

In [23]:
# print(dataset[0][0]['user']['channel'][150][0][3])

count = 0
for h in dataset[0][0]['user']['channel'][100][0]:
#     h = h.squeeze(0)
    h_real = h.real
    h_imag = h.imag
    if np.sum(np.abs(h_real)) ==0:
        count+=1
    elif np.sum(np.abs(h_imag)) == 0:
        count+=1

print("0이 존재하는 채널 개수",count)

0이 존재하는 채널 개수 0


In [24]:
import numpy as np

# 1) (user, ue_port, bs_ant, subc) → (bs_ant, subc) 로 squeeze
H = dataset[0][0]['user']['channel'][100, 0]   # shape: (32, 64), complex

# 2) BS 안테나 인덱스 3의 서브캐리어 벡터 (64,)
print("Antenna #3 subcarriers:", H[3])

# 3) 전체 서브캐리어(32×64) 중 값이 정확히 0인 요소 개수
zero_elements = np.sum(H == 0)
print("0+0j인 서브캐리어 개수:", zero_elements)

# 4) 서브캐리어 전부가 0인 안테나 포트(행) 개수
zero_ports = np.sum(np.all(H == 0, axis=1))
print("완전 0+0j 안테나 포트 개수:", zero_ports)

# 5) 만약 “값이 하나도 0이 아닌” 서브캐리어 요소 개수를 보고 싶다면
nonzero_elements = np.sum(np.abs(H) > 0)
print("0이 아닌 서브캐리어 개수:", nonzero_elements)


Antenna #3 subcarriers: [ 5.3233252e-06-8.73100362e-06j  4.7875683e-06-9.03589535e-06j
  4.2344141e-06-9.30795068e-06j  3.6658719e-06-9.54618190e-06j
  3.0840083e-06-9.74972318e-06j  2.4909377e-06-9.91783418e-06j
  1.8888149e-06-1.00499046e-05j  1.2798286e-06-1.01454543e-05j
  6.6619128e-07-1.02041367e-05j  5.0133124e-08-1.02257372e-05j
 -5.6610725e-07-1.02101776e-05j -1.1802904e-06-1.01575160e-05j
 -1.7901845e-06-1.00679417e-05j -2.3935731e-06-9.94178117e-06j
 -2.9882635e-06-9.77949367e-06j -3.5720950e-06-9.58166765e-06j
 -4.1429457e-06-9.34902255e-06j -4.6987411e-06-9.08240327e-06j
 -5.2374617e-06-8.78277933e-06j -5.7571492e-06-8.45123941e-06j
 -6.2559161e-06-8.08898767e-06j -6.7319493e-06-7.69734197e-06j
 -7.1835188e-06-7.27772431e-06j -7.6089841e-06-6.83165945e-06j
 -8.0067985e-06-6.36076902e-06j -8.3755176e-06-5.86676424e-06j
 -8.7137996e-06-5.35144000e-06j -9.0204167e-06-4.81666848e-06j
 -9.2942537e-06-4.26439374e-06j -9.5343166e-06-3.69662257e-06j
 -9.7397324e-06-3.11541817e-06j

## 결측치 제거 및 dataload

In [25]:
# ─────────────────────────────────────────────
# ❶ IterableDataset: 모든 유저·서브캐리어를 스트리밍
import torch
from torch.utils.data import IterableDataset, DataLoader
import numpy as np

class ChannelSeqDataset(IterableDataset):
    """
    • seq_len 개의 과거 채널 벡터(real 64 + imag 64 → 128) → 다음 시점 벡터 예측
    • 벡터는 평균전력 1 로 power‑normalize 후 반환
    """
    def __init__(self, scenes, seq_len: int = 5, eps: float = 1e-9):
        super().__init__()
        self.scenes   = scenes
        self.seq_len  = seq_len
        self.eps      = eps                        # 0 division 방지용 신호세기의 크기 
        ch0           = scenes[0][0]['user']['channel']
        self.U        = ch0.shape[0]               # 사용자 수
        self.A        = ch0.shape[2]               # 안테나 32
        self.S        = ch0.shape[3]               # 서브캐리어 64
        self.vec_len  = 2 * self.A                 # 64 real + imag
        0
    def _vec(self, scene, u: int, sc: int) -> torch.Tensor:
        """(32,) complex → (64,) float32  +  power norm"""
        h = scene[0]['user']['channel'][u, 0, :, sc]          # (32,)
        v = np.concatenate([h.real, h.imag]).astype(np.float32)
        p = np.mean(v * v) + self.eps                         # 평균 전력: 채널 벡터 h의 각 성분의 진폭 제곱을 합산
        v /= np.sqrt(p)                                       # 정규화
        return torch.from_numpy(v)                            # (64,)

    def __iter__(self):
        T = len(self.scenes)
        for t in range(self.seq_len, T):                      # 타깃 시점
            past_scenes = self.scenes[t - self.seq_len : t]
            tgt_scene   = self.scenes[t]
            for u in range(self.U):
                for s in range(self.S):
                    seq = torch.stack([self._vec(ps, u, s) for ps in past_scenes])
                    if not torch.any(seq):                    # 전부 0 이면 skip
                        continue
                    target     = self._vec(tgt_scene, u, s)
                    if not torch.any(target): # target이 0이면 스킵
                        continue
                    masked_pos = torch.tensor([self.seq_len - 2], dtype=torch.long)
                    yield seq, masked_pos, target             # shapes: (5,64) / (1,) / (64,)
    
    def __len__(self):
         return (len(self.scenes) - self.seq_len) * self.U * self.S
# ─────────────────────────────────────────────
# ❷ 학습·검증 DataLoader train : val = 6 : 4
seq_len      = 5
split_ratio  = 0.6
split_idx    = int(len(dataset) * split_ratio)

train_ds = ChannelSeqDataset(dataset[:split_idx], seq_len=seq_len)
val_ds   = ChannelSeqDataset(dataset[split_idx:], seq_len=seq_len)

# train_ds 순회하면서 feature/target min, max 계산


batch_size   = 32
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
# ─────────────────────────────────────────────


In [26]:
from sklearn.preprocessing   import MinMaxScaler
from torch.utils.data        import TensorDataset, DataLoader

# ❶ 모든 seq/mask/target을 한 번에 뽑아서 NumPy로
X_list, mposes_list, y_list = [], [], []
for seq, mpos, tgt in ChannelSeqDataset(dataset, seq_len=seq_len):
    X_list.append( seq.numpy() )           # (seq_len, vec_len)
    mposes_list.append( mpos.item() )      # 스칼라 마스크 인덱스
    y_list.append( tgt.numpy() )           # (vec_len,)

X       = np.stack(X_list,      axis=0)    # (N, seq_len, vec_len)
mposes  = np.array(mposes_list).reshape(-1,1)  # (N, 1)
y       = np.stack(y_list,      axis=0)    # (N, vec_len)

# ❷ Train/Val split  (6:4)
split_idx = int(len(X) * split_ratio)
X_tr,  X_va  = X[:split_idx],  X[split_idx:]
mp_tr, mp_va = mposes[:split_idx], mposes[split_idx:]
y_tr,  y_va  = y[:split_idx],  y[split_idx:]

# ❸ 스케일링 (X, y 만)
scaler_x = MinMaxScaler(); scaler_y = MinMaxScaler()
Ntr, L, D = X_tr.shape

# train 에서만 fit
scaler_x .fit( X_tr.reshape(-1, D) )
scaler_y .fit( y_tr )

# transform 후 원래 shape 복원
X_tr_s = scaler_x.transform(X_tr.reshape(-1, D)).reshape(Ntr, L, D)
X_va_s = scaler_x.transform(X_va.reshape(-1, D)).reshape(X_va.shape)
y_tr_s = scaler_y.transform(y_tr)
y_va_s = scaler_y.transform(y_va)

# ❹ TensorDataset 에 세 개 모두 담기
train_ds = TensorDataset(
    torch.from_numpy(X_tr_s).float(),     # seqs
    torch.from_numpy(mp_tr).long(),       # mposes
    torch.from_numpy(y_tr_s).float()      # tgts
)
val_ds   = TensorDataset(
    torch.from_numpy(X_va_s).float(),
    torch.from_numpy(mp_va).long(),
    torch.from_numpy(y_va_s).float()
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)

# 이제 배치 언패킹이 딱 맞습니다!
first_batch = next(iter(train_loader))
seqs, mposes, tgts = first_batch   # (B, L, D), (B,1), (B, D)


In [27]:
len(train_ds) #4x727x64

200563

In [28]:
len(val_ds) #1x727x64

133709

In [29]:
# 1) DataLoader 설정 확인
print(train_loader)                # DataLoader 정보 전체
print("batch_size:", train_loader.batch_size)
print("dataset:",   train_loader.dataset)

# 총 샘플 수
print("total samples:", len(train_loader.dataset))
# → (len(scenes) - seq_len) * U * S 와 동일한 값

# 총 배치 수
print("total batches:", len(train_loader))
# → ceil(total_samples / batch_size)


# 3) 첫 번째 배치 내용 확인
first_batch = next(iter(train_loader))
seqs, mposes, tgts = first_batch
print("seqs.shape:",   seqs.shape)    # (B, seq_len, vec_len)
print("mposes.shape:", mposes.shape)  # (B, 1)
print("tgts.shape:",   tgts.shape)    # (B, vec_len)


batch_size: 32
dataset: <torch.utils.data.dataset.TensorDataset object at 0x0000015E8A42D640>
total samples: 200563
total batches: 6268
seqs.shape: torch.Size([32, 5, 64])
mposes.shape: torch.Size([32, 1])
tgts.shape: torch.Size([32, 64])


In [30]:
# 1) train_ds 자체 정보 출력
print(train_ds)
print("전체 샘플 수 (len):", len(train_loader))

# 2) 앞에서 5개 예시 뽑아서 확인
for idx, (seq, mpos, tgt) in enumerate(train_loader):
    print(f"\n샘플 #{idx}")
    print("  seq shape :", seq.shape)    # (seq_len, feat_dim)
    print("  masked_pos :", mpos)        # tensor([ ... ])
    print("  target shape:", tgt.shape)   # (feat_dim,)
    print("  seq example:\n", seq)        # 실제 값 보기
    print("  target example:\n", tgt)
    if idx >= 4:
        break


전체 샘플 수 (len): 6268

샘플 #0
  seq shape : torch.Size([32, 5, 64])
  masked_pos : tensor([[3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3],
        [3]])
  target shape: torch.Size([32, 64])
  seq example:
 tensor([[[0.4833, 0.4948, 0.4946,  ..., 0.5279, 0.5228, 0.5068],
         [0.5166, 0.5268, 0.5256,  ..., 0.4972, 0.4935, 0.4776],
         [0.5098, 0.5216, 0.5219,  ..., 0.5076, 0.5024, 0.4853],
         [0.4750, 0.4875, 0.4882,  ..., 0.5366, 0.5304, 0.5138],
         [0.5198, 0.5287, 0.5263,  ..., 0.4948, 0.4924, 0.4776]],

        [[0.4616, 0.4835, 0.4930,  ..., 0.4854, 0.4837, 0.4695],
         [0.5038, 0.5247, 0.5335,  ..., 0.4501, 0.4515, 0.

## 이론적 input_size

In [31]:
# (1) 장면(scene) 수, 시퀀스 길이
T = len(dataset)
L = 5          # 이제 int!

# (2) 사용자 수(U), 서브캐리어 수(S)
ch0 = dataset[0][0]['user']['channel']  # shape = (U,1,A,S)
U   = ch0.shape[0]
S   = ch0.shape[3]

# (3) 이론상 최대 시퀀스 수
total_possible = (T - L) * U * S

# (4) train/val 분할 기준에서 이론상 최대 개수
train_max = int(total_possible * split_ratio)
val_max   = total_possible - train_max

print(f"▶ 이론상 전체 가능 시퀀스: {total_possible}")
print(f"▶ 이론상 Train 최대 샘플: {train_max}")
print(f"▶ 이론상 Val   최대 샘플: {val_max}")


▶ 이론상 전체 가능 시퀀스: 465280
▶ 이론상 Train 최대 샘플: 279168
▶ 이론상 Val   최대 샘플: 186112


In [32]:
import numpy as np

# 전체 실제 샘플 수
N_total   = X.shape[0]
# 실제 train/val 샘플 수
N_train   = X_tr.shape[0]
N_val     = X_va.shape[0]

# 실제 배치 수 (올림)
bsize     = batch_size
B_train   = int(np.ceil(N_train / bsize))
B_val     = int(np.ceil(N_val   / bsize))

print(f"▶ 실제 전체 샘플 수: {N_total}")
print(f"▶ 실제   Train 샘플 수: {N_train}   → 배치 수: {B_train}")
print(f"▶ 실제   Val   샘플 수: {N_val}   → 배치 수: {B_val}")


▶ 실제 전체 샘플 수: 334272
▶ 실제   Train 샘플 수: 200563   → 배치 수: 6268
▶ 실제   Val   샘플 수: 133709   → 배치 수: 4179


# 아래 코드 구조
┌──────────────────────────────────────────────────────────────┐
│ input_ids  (B, seq_len, element_length)  ─┐                 │
│ masked_pos (B, num_mask)                  ├─>  LWM backbone │
│                                           │    (12-층 트랜스포머)  
└────────────────────────────────────────────┘         │
            logits_lm  (B, num_mask, element_length)  │   enc_output (B, seq_len, d_model)
                                                      ▼
                        ┌─[풀링]───────────────┐      ←── feat (B, d_model)
                        │ 첫 토큰(0번) 선택    │
                        │   or 평균/최대 풀링 │
                        └──────────────────────┘
                                      ▼
                       FC 헤드  (d_model → hidden_dim → out_dim)
                                      ▼
                                out (B, out_dim)

# 시각적비유

[패치 프로젝터]──▶[Transformer ×12]──▶[LayerNorm]──┐
                                                  ├─▶ 64-차 벡터 (CLS 또는 풀링) ─▶ MLP ─▶ out                                                
[Positional Embedding]─────────────────────────────┘


In [33]:
"""
LWMWithHead: 사전학습된 LWM(Transformer encoder)을 ‘백본(backbone)’으로 사용하고,
             그 뒤에 새로운 완전연결(FC) 헤드(head)를 붙여 다운스트림 작업
             (회귀·분류 등)에 사용할 수 있도록 만든 래퍼(wrapper) 클래스입니다.

변경점:
- input_dim: 실제 데이터 차원 (예: 64)
- patch_length: backbone이 기대하는 패치 길이 (예: 16)
- 기존 element_length 파라미터 대신 위 두 개로 명확히 분리
- forward()에서 투영 레이어(self.input_proj) 적용
"""

import torch
import torch.nn as nn
from lwm_model import lwm   # 기존 LWM 모델 클래스

class LWMWithHead(nn.Module):
    def __init__(
        self,
        input_dim:      int,    # 실제 입력 벡터 차원, 예: 64 (real+imag 합친 길이)
        patch_length:   int,    # backbone in_features, 예: 16 (사전학습된 패치 길이)
        d_model:        int = 64,# backbone hidden size (사전학습된 d_model)
        max_len:        int = 129,# 포지셔널 인코딩 최대 길이
        n_layers:       int = 12,# transformer encoder 층 수
        hidden_dim:     int = 256,# 헤드 FC 중간 차원
        out_dim:        int = 64, # 최종 출력 차원
        freeze_backbone: bool = True,  # 백본 동결 여부
        ckpt_path:      str | None = "./model_weights.pth", 
        device:         str = "cuda",
    ):
        super().__init__()

        # ────────────────────────────────────
        # ➊ 입력 투영 레이어
        #   • 실제 데이터(input_dim) → patch_length 차원으로 줄여줌
        #   • backbone이 기대하는 in_features 길이에 맞추기 위함
        # ────────────────────────────────────
        self.input_proj = nn.Linear(input_dim, patch_length)

        # ────────────────────────────────────
        # ➋ 백본(backbone) 초기화
        # ────────────────────────────────────
        if ckpt_path is None:
            # 랜덤 초기화된 backbone
            self.backbone = lwm(
                element_length=patch_length,
                d_model=d_model,
                max_len=max_len,
                n_layers=n_layers
            ).to(device)
        else:
            # 사전학습 가중치 로드
            self.backbone = lwm.from_pretrained(
                ckpt_name=ckpt_path,
                device=device
            )
        # 백본 동결(gradient off)
        if freeze_backbone:
            for p in self.backbone.parameters():
                p.requires_grad = False

        # ────────────────────────────────────
        # ➌ 다운스트림 FC 헤드 정의
        #   • d_model → hidden_dim → out_dim
        # ────────────────────────────────────
        self.head = nn.Sequential(
            nn.Linear(d_model, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_dim)
        )

    def forward(self, input_ids, masked_pos):
        """
        Parameters
        ----------
        input_ids : Tensor  (B, seq_len, input_dim)
            원본 채널 시퀀스, 예: (B, 5, 64)
        masked_pos: Tensor  (B, num_mask)
            backbone의 masked position 인덱스

        Returns
        -------
        out : Tensor  (B, out_dim)
            최종 FC 헤드를 거친 예측값
        """
        # ────────────────────────────────────
        # 1) 입력 투영: (B, L, 64) → (B, L, 16)
        # ────────────────────────────────────
        x = self.input_proj(input_ids)

        # ────────────────────────────────────
        # 2) backbone forward
        #    • logits_lm (unused), enc_output: (B, L, d_model)
        # ────────────────────────────────────
        _, enc_output = self.backbone(x, masked_pos)

        # ────────────────────────────────────
        # 3) 특징 추출: CLS 토큰 벡터 (첫 번째 토큰)
        #    • feat: (B, d_model)
        # ────────────────────────────────────
        feat = enc_output[:, 0, :]

        # ────────────────────────────────────
        # 4) FC 헤드 통과
        #    • out: (B, out_dim)
        # ────────────────────────────────────
        out = self.head(feat)
        return out


## input_size
-input_size = (scene - seq_len) * U * S -> (10-5)+69040*64 = 22092800  
-batch_size = 32  
-배치 수 = input_size / batch_size = 690400배치

In [34]:
X_tr_s.shape

(200563, 5, 64)

In [35]:
# ─────────────────────────────────────────────
# 모델 초기화 (새로운 생성자 시그니처에 맞춰 수정)
# ─────────────────────────────────────────────
from torch.optim import Adam

batch_size, seq_len, D = X_tr_s.shape  # D = 64
hidden_dim = 256

model = LWMWithHead(
    input_dim     = D,      # 실제 채널 벡터 차원 (real+imag=64)
    patch_length  = 16,     # pretrained 백본이 기대하는 패치 길이
    d_model       = 64,     # pretrained 백본 hidden size
    max_len       = seq_len,
    n_layers      = 12,
    hidden_dim    = hidden_dim,
    out_dim       = D,      # downstream 예측 차원 (64)
    freeze_backbone = False,
    ckpt_path     = "./model_weights.pth",
    device        = device
).to(device)
# ─────────────────────────────────────────────

# 손실함수 & 옵티마이저는 그대로
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-4)


Model loaded successfully from ./model_weights.pth to cuda


In [36]:
# model: LWMWithHead 인스턴스
print("Backbone requires_grad flags:")
for name, param in model.backbone.named_parameters():
    print(f"  {name:40s}: {param.requires_grad}")

# 또한 전체가 동결됐는지 한 줄로 요약하려면:
all_frozen = all(not p.requires_grad for p in model.backbone.parameters())
print(f"\n→ Is backbone fully frozen? {all_frozen}")


Backbone requires_grad flags:
  decoder_bias                            : True
  embedding.proj.weight                   : True
  embedding.proj.bias                     : True
  embedding.pos_embed.weight              : True
  embedding.norm.alpha                    : True
  embedding.norm.bias                     : True
  layers.0.enc_self_attn.W_Q.weight       : True
  layers.0.enc_self_attn.W_Q.bias         : True
  layers.0.enc_self_attn.W_K.weight       : True
  layers.0.enc_self_attn.W_K.bias         : True
  layers.0.enc_self_attn.W_V.weight       : True
  layers.0.enc_self_attn.W_V.bias         : True
  layers.0.enc_self_attn.linear.weight    : True
  layers.0.enc_self_attn.linear.bias      : True
  layers.0.enc_self_attn.norm.alpha       : True
  layers.0.enc_self_attn.norm.bias        : True
  layers.0.pos_ffn.fc1.weight             : True
  layers.0.pos_ffn.fc1.bias               : True
  layers.0.pos_ffn.fc2.weight             : True
  layers.0.pos_ffn.fc2.bias            

In [37]:
# 모델 평가 함수
import torch
import torch.nn.functional as F

# ─────────────────────────────────────────
# 1. 배치 단위 RMSE, NMSE 함수
# ─────────────────────────────────────────
def rmse(pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
    """
    Root-Mean-Squared Error
    returns: 스칼라 (배치 평균)
    """
    return torch.sqrt(F.mse_loss(pred, target, reduction="mean"))   # √MSE

def nmse(pred: torch.Tensor, target: torch.Tensor, eps : float = 1e-12) -> torch.Tensor:
    """
    Normalized MSE  =  E[‖ŷ − y‖²] / E[‖y‖²]
      · 채널 예측 분야에서 흔히 쓰는 지표
    returns: 스칼라 (배치 평균)
    """
    # (B, …) → (B,)  : 각 샘플별 제곱합
    mse_per_sample   = ((pred - target)**2).view(pred.size(0), -1).sum(dim=1)
    power_per_sample = (target**2).view(target.size(0), -1).sum(dim=1) + eps
    return (mse_per_sample / power_per_sample).mean()


# ─────────────────────────────────────────
# 2. 검증 루프 예시
# ─────────────────────────────────────────
def evaluate(model, loader, device="cuda"):
    """
    Validation loop for IterableDataset.
    Returns average RMSE and NMSE over all samples.
    """
    model.eval()
    total_rmse, total_nmse, total_samples = 0.0, 0.0, 0

    with torch.no_grad():
        for input_ids, masked_pos, target in loader:
            # Move to device
            input_ids, masked_pos, target = (
                input_ids.to(device),
                masked_pos.to(device),
                target.to(device),
            )
            # Batch size
            bs = input_ids.size(0)

            # Forward
            pred = model(input_ids, masked_pos)

            # Accumulate batch metrics
            total_rmse    += rmse(pred, target).item() * bs
            total_nmse    += nmse(pred, target).item() * bs
            total_samples += bs

    # Compute averages
    return {
        "RMSE": total_rmse / total_samples,
        "NMSE": total_nmse / total_samples
    }



# model training

In [38]:
from tqdm import tqdm
import time, torch

num_epochs   = 10                      # 원하는 epoch 수
start_time   = time.time()

for epoch in range(1, num_epochs+1):

    # ① TRAIN ────────────────────────────────
    model.train()
    run_loss = 0.0

    tq = tqdm(train_loader, desc=f"[{epoch:02d}/{num_epochs}] train", leave=False)
    for b, (inp, mpos, tgt) in enumerate(tq, 1):
        inp, mpos, tgt = inp.to(device), mpos.to(device), tgt.to(device)

        optimizer.zero_grad()
        pred  = model(inp, mpos).squeeze(-1)
        loss  = criterion(pred, tgt)
        loss.backward()
        optimizer.step()

        run_loss += loss.item()
        if b % 100 == 0:                   # 100 배치마다 진행바에 표시
            tq.set_postfix(loss=run_loss/b)

    avg_train_loss = run_loss / b

    # ② VALID ────────────────────────────────
    metrics   = evaluate(model, val_loader, device)
    val_rmse  = metrics["RMSE"]
    val_nmse  = metrics["NMSE"]
    val_nmse_db = 10 * torch.log10(torch.tensor(val_nmse)).item()

    print(
    f"[{epoch:02d}/{num_epochs}] "
    f"Train Loss: {avg_train_loss:.4f}  "
    f"Val RMSE: {val_rmse:.4f}  "
    f"Val NMSE: {val_nmse:.4e}  "
    f"Val NMSE_dB: {val_nmse_db:.1f} dB"
)
print(f"Total training time: {time.time() - start_time:.2f}s")

[01/10] Train Loss: 0.0067  Val RMSE: 0.0420  Val NMSE: 8.9593e-03  Val NMSE_dB: -20.5 dB


[02/10] Train Loss: 0.0023  Val RMSE: 0.0404  Val NMSE: 8.3298e-03  Val NMSE_dB: -20.8 dB


[03/10] Train Loss: 0.0021  Val RMSE: 0.0384  Val NMSE: 7.7766e-03  Val NMSE_dB: -21.1 dB


[04/10] Train Loss: 0.0020  Val RMSE: 0.0379  Val NMSE: 7.5562e-03  Val NMSE_dB: -21.2 dB


[05/10] Train Loss: 0.0019  Val RMSE: 0.0374  Val NMSE: 7.4702e-03  Val NMSE_dB: -21.3 dB


[06/10] Train Loss: 0.0018  Val RMSE: 0.0368  Val NMSE: 7.3053e-03  Val NMSE_dB: -21.4 dB


[07/10] Train Loss: 0.0018  Val RMSE: 0.0365  Val NMSE: 7.2069e-03  Val NMSE_dB: -21.4 dB


[08/10] Train Loss: 0.0017  Val RMSE: 0.0368  Val NMSE: 7.3255e-03  Val NMSE_dB: -21.4 dB


[09/10] Train Loss: 0.0016  Val RMSE: 0.0361  Val NMSE: 7.1586e-03  Val NMSE_dB: -21.5 dB


[10/10] Train Loss: 0.0016  Val RMSE: 0.0361  Val NMSE: 7.1582e-03  Val NMSE_dB: -21.5 dB
Total training time: 4206.82s
